#  PySpark-SQL Project nº2


### What is the accumulated number of taxi trips per month?

In [90]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    
    #Create dataframe
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips")
    
    #Data frame created from SQL statement
    tripsbymonthDF = spark.sql('SELECT SUBSTRING(trip_start_time, 1, 2) AS trip_month, COUNT(*) AS total_trips FROM trips \
    WHERE trip_start_time!="" GROUP BY trip_month ORDER BY trip_month ')
    tripsbymonthDF.collect()
    sc.stop()
except:
    sc.stop()


CPU times: user 21.3 ms, sys: 43.2 ms, total: 64.5 ms
Wall time: 3.37 s


###  For each pickup region, report the list of unique drop-off regions

In [89]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    #Create dataframe
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips")
    #Data frame created from SQL statement
    tripsbymonthDF = spark.sql('SELECT pickup_ID, COLLECT_SET(dropoff_ID) AS list_of_dropoffs FROM \
    trips WHERE pickup_ID!="" AND dropoff_ID!=""   GROUP BY pickup_ID ORDER BY pickup_ID')
    tripsbymonthDF.collect()
except:
    sc.stop()


CPU times: user 35.8 ms, sys: 47.9 ms, total: 83.7 ms
Wall time: 2.39 s


### What is the expected charge/cost of a taxi ride, given the pickup region ID, the weekday (0 = Monday, 6 = Sunday) and time in format “hour AM/PM”?

Concatenated Key with simple GROUP BY

In [79]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    #Create dataframe
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips") 
    
    #Create View to do some pre-processing
    spark.sql('DROP VIEW IF EXISTS correct_prices')
    correct_pricesDF= spark.sql('CREATE TEMPORARY VIEW correct_prices AS SELECT trip_start_time,pickup_ID,\
    CASE WHEN (fare="") THEN 0 ELSE REPLACE(fare, ",", "") END AS fare, \
    CASE WHEN (tips="") THEN 0 ELSE REPLACE(tips, ",", "") END AS tips, \
    CASE WHEN (tolls="") THEN 0 ELSE REPLACE(tolls, ",", "") END AS tolls, \
    CASE WHEN (extras="") THEN 0 ELSE REPLACE(extras, ",", "") END AS extras ,\
    CASE WHEN (trip_total="") THEN 0 ELSE REPLACE(trip_total, ",", "") END AS trip_total \
    FROM trips')
    
    #Data frame created from SQL statement
    tripsbymonthDF = spark.sql('SELECT  \
    CONCAT(pickup_ID,"_",WEEKDAY(to_timestamp(trip_start_time,"MM/dd/yyyy hh:mm:ss aa")),\
    "_",SUBSTRING(trip_start_time,12,2),SUBSTRING(trip_start_time,-2,2))AS KEY, \
    ROUND(AVG(CASE WHEN (fare+tips+tolls+extras!=trip_total AND fare+tips+tolls+extras!=0) THEN fare+tips+tolls+extras\
    ELSE trip_total END),2) AS Cost \
    FROM correct_prices\
    WHERE (trip_total!=0 OR fare+tips+tolls+extras!=0) AND pickup_ID!="" \
    GROUP BY KEY\
    ORDER BY Cost DESC')
    
    tripsbymonthDF.show(tripsbymonthDF.count())
    sc.stop()
except:
    sc.stop()


+------------------+-------+
|               KEY|   Cost|
+------------------+-------+
|17031832200_4_10AM|2051.08|
|17031080202_0_08PM| 513.56|
|17031071200_4_11PM| 482.87|
|17031081600_1_11AM| 443.76|
|17031062100_6_01AM| 151.41|
|17031242200_6_12PM| 141.95|
|17031161100_0_03PM|  120.0|
|17031081201_0_07PM| 119.67|
|17031980100_4_08PM|  97.82|
|17031050600_4_01PM|  97.05|
|17031081700_3_03PM|  90.04|
|17031160200_5_08AM|   90.0|
|17031841900_2_02AM|  82.25|
|17031980000_0_12PM|  80.52|
|17031836200_3_06PM|  77.25|
|17031830700_2_06PM|  77.05|
|17031839200_4_04PM|   75.0|
|17031980000_3_03AM|  74.77|
|17031836200_5_02PM|   72.9|
|17031031400_5_02PM|  72.63|
|17031804603_2_05PM|   72.3|
|17031812400_4_11AM|   71.0|
|17031071200_2_06PM|  70.85|
|17031832000_2_07PM|  70.63|
|17031030900_3_08AM|  70.45|
|17031160300_0_07PM|   70.0|
|17031830700_0_05PM|  69.25|
|17031980000_6_04AM|  68.55|
|17031842300_0_05AM|  68.25|
|17031081401_3_11AM|  67.99|
|17031030200_6_06AM|  67.92|
|17031330100_3

Complex Group By

In [80]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    
    #Create dataframe
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips") 
    
    #Create View to do some pre-processing
    spark.sql('DROP VIEW IF EXISTS correct_prices')
    correct_pricesDF= spark.sql('CREATE TEMPORARY VIEW correct_prices AS SELECT trip_start_time,pickup_ID,\
    CASE WHEN (fare="") THEN 0 ELSE REPLACE(fare, ",", "") END AS fare, \
    CASE WHEN (tips="") THEN 0 ELSE REPLACE(tips, ",", "") END AS tips, \
    CASE WHEN (tolls="") THEN 0 ELSE REPLACE(tolls, ",", "") END AS tolls, \
    CASE WHEN (extras="") THEN 0 ELSE REPLACE(extras, ",", "") END AS extras ,\
    CASE WHEN (trip_total="") THEN 0 ELSE REPLACE(trip_total, ",", "") END AS trip_total \
    FROM trips')
 
    #Data frame created from SQL statement
    tripsbymonthDF = spark.sql('SELECT  \
    pickup_ID,WEEKDAY(to_timestamp(trip_start_time,"MM/dd/yyyy hh:mm:ss aa")) as weekday,\
    CONCAT(SUBSTRING(trip_start_time,12,2),SUBSTRING(trip_start_time,-2,2))AS hour, \
    ROUND(AVG(CASE WHEN (fare+tips+tolls+extras!=trip_total AND fare+tips+tolls+extras!=0) THEN fare+tips+tolls+extras\
    ELSE trip_total END),2) AS Cost \
    FROM correct_prices\
    WHERE (trip_total!=0 OR fare+tips+tolls+extras!=0) AND pickup_ID!="" \
    GROUP BY pickup_ID, weekday, hour\
    ORDER BY Cost DESC')
    
    tripsbymonthDF.show(tripsbymonthDF.count())
    sc.stop()
except:
    sc.stop()

+-----------+-------+----+-------+
|  pickup_ID|weekday|hour|   Cost|
+-----------+-------+----+-------+
|17031832200|      4|10AM|2051.08|
|17031080202|      0|08PM| 513.56|
|17031071200|      4|11PM| 482.87|
|17031081600|      1|11AM| 443.76|
|17031062100|      6|01AM| 151.41|
|17031242200|      6|12PM| 141.95|
|17031161100|      0|03PM|  120.0|
|17031081201|      0|07PM| 119.67|
|17031980100|      4|08PM|  97.82|
|17031050600|      4|01PM|  97.05|
|17031081700|      3|03PM|  90.04|
|17031160200|      5|08AM|   90.0|
|17031841900|      2|02AM|  82.25|
|17031980000|      0|12PM|  80.52|
|17031836200|      3|06PM|  77.25|
|17031830700|      2|06PM|  77.05|
|17031839200|      4|04PM|   75.0|
|17031980000|      3|03AM|  74.77|
|17031836200|      5|02PM|   72.9|
|17031031400|      5|02PM|  72.63|
|17031804603|      2|05PM|   72.3|
|17031812400|      4|11AM|   71.0|
|17031071200|      2|06PM|  70.85|
|17031832000|      2|07PM|  70.63|
|17031030900|      3|08AM|  70.45|
|17031160300|      0


### Additional question: Present the most popular short routes and long routes, with the number of trips and the percentage paid by cash and by credit card.


Solution not using JOINS

In [63]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    #Create Dataframe
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips") 
    
    #Data frame created from SQL statement
    longtripsDF = spark.sql('SELECT  \
    pickup_ID,dropoff_ID, Count(*) AS Total ,\
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Cash") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Cash, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Credit Card") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Credit_Card \
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID\
    HAVING AVG(trip_miles)>=10\
    ORDER BY Total DESC')
    #Data frame created from SQL statement
    shorttripsDF = spark.sql('SELECT  \
    pickup_ID,dropoff_ID, Count(*) AS Total ,\
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Cash") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Cash, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Credit Card") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Credit_Card \
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID\
    HAVING AVG(trip_miles)<10\
    ORDER BY Total DESC')
    
    longtripsDF.show(10)
    
    shorttripsDF.show(10)
    
    sc.stop()
except:
    sc.stop()

+-----------+-----------+-----+-----+-----------+
|  pickup_ID| dropoff_ID|Total| Cash|Credit_Card|
+-----------+-----------+-----+-----+-----------+
|17031980000|17031839100| 1496|21.0%|      78.0%|
|17031980000|17031320100| 1295|25.0%|      75.0%|
|17031839100|17031980000| 1200|25.0%|      74.0%|
|17031980000|17031081500| 1075|30.0%|      69.0%|
|17031320100|17031980000| 1060|30.0%|      69.0%|
|17031081500|17031980000|  854|37.0%|      63.0%|
|17031980000|17031081403|  746|27.0%|      73.0%|
|17031980000|17031081700|  522|24.0%|      75.0%|
|17031081403|17031980000|  465|33.0%|      66.0%|
|17031980000|17031081201|  441|37.0%|      61.0%|
+-----------+-----------+-----+-----+-----------+
only showing top 10 rows

+-----------+-----------+-----+------------------+-----------+
|  pickup_ID| dropoff_ID|Total|              Cash|Credit_Card|
+-----------+-----------+-----+------------------+-----------+
|17031839100|17031839100| 4124|             63.0%|      36.0%|
|17031320100|170318391

Compare time with last work

In [83]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips") 

    tripsDF = spark.sql('SELECT  \
    pickup_ID,dropoff_ID, Count(*) AS Total ,\
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Cash") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Cash, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Credit Card") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Credit_Card \
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID\
    ORDER BY Total DESC')
    
    
    tripsDF.show(tripsDF.count())

    
    sc.stop()
except:
    sc.stop()

+-----------+-----------+-----+-------------------+-------------------+
|  pickup_ID| dropoff_ID|Total|               Cash|        Credit_Card|
+-----------+-----------+-----+-------------------+-------------------+
|17031839100|17031839100| 4124|              63.0%|              36.0%|
|17031320100|17031839100| 2784| 55.00000000000001%|              44.0%|
|17031839100|17031320100| 2552| 56.99999999999999%|              42.0%|
|17031081500|17031839100| 1960| 56.00000000000001%|              43.0%|
|17031839100|17031281900| 1916|              67.0%|              32.0%|
|17031281900|17031839100| 1890|              64.0%|              35.0%|
|17031839100|17031081500| 1626| 57.99999999999999%|              41.0%|
|17031839100|17031081700| 1609| 55.00000000000001%|              44.0%|
|17031081700|17031839100| 1503| 56.99999999999999%|              42.0%|
|17031980000|17031839100| 1496|              21.0%|              78.0%|
|17031839100|17031081800| 1422|              52.0%|             

Solution with joins

In [8]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips")
    
    spark.sql('DROP VIEW IF EXISTS count')
    countDF = spark.sql(' CREATE TEMPORARY VIEW count AS SELECT  \
    pickup_ID,dropoff_ID, Count(*) AS Total, AVG(trip_miles) AS avg_miles \
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    spark.sql('DROP VIEW IF EXISTS cash')
    cashDF=spark.sql('CREATE TEMPORARY VIEW cash AS SELECT  \
    pickup_ID,dropoff_ID, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Cash") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Cash\
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    spark.sql('DROP VIEW IF EXISTS credit_card')
    credit_cardDF=spark.sql('CREATE TEMPORARY VIEW credit_card AS SELECT  \
    pickup_ID,dropoff_ID, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Credit Card") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Credit_Card\
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    resultlongDF=spark.sql('SELECT A.pickup_ID, A.dropoff_ID, A.Total, B.Cash, C.Credit_Card FROM count as A JOIN cash AS B \
    ON A.pickup_ID=B.pickup_ID AND A.dropoff_ID=B.dropoff_ID JOIN credit_card AS C \
    ON A.pickup_ID=C.pickup_ID AND A.dropoff_ID=C.dropoff_ID WHERE A.avg_miles>=10  ORDER BY A.Total DESC')
    
    resultshortDF=spark.sql('SELECT A.pickup_ID, A.dropoff_ID, A.Total, B.Cash, C.Credit_Card FROM count as A JOIN cash AS B \
    ON A.pickup_ID=B.pickup_ID AND A.dropoff_ID=B.dropoff_ID JOIN credit_card AS C \
    ON A.pickup_ID=C.pickup_ID AND A.dropoff_ID=C.dropoff_ID  WHERE A.avg_miles < 10 ORDER BY A.Total DESC')
    
    resultlongDF.show(10)
    
    resultshortDF.show(10)
    
    

    sc.stop()
except:
    sc.stop()

+-----------+-----------+-----+-----+-----------+
|  pickup_ID| dropoff_ID|Total| Cash|Credit_Card|
+-----------+-----------+-----+-----+-----------+
|17031980000|17031839100| 1496|21.0%|      78.0%|
|17031980000|17031320100| 1295|25.0%|      75.0%|
|17031839100|17031980000| 1200|25.0%|      74.0%|
|17031980000|17031081500| 1075|30.0%|      69.0%|
|17031320100|17031980000| 1060|30.0%|      69.0%|
|17031081500|17031980000|  854|37.0%|      63.0%|
|17031980000|17031081403|  746|27.0%|      73.0%|
|17031980000|17031081700|  522|24.0%|      75.0%|
|17031081403|17031980000|  465|33.0%|      66.0%|
|17031980000|17031081201|  441|37.0%|      61.0%|
+-----------+-----------+-----+-----+-----------+
only showing top 10 rows

+-----------+-----------+-----+------------------+-----------+
|  pickup_ID| dropoff_ID|Total|              Cash|Credit_Card|
+-----------+-----------+-----+------------------+-----------+
|17031839100|17031839100| 4124|             63.0%|      36.0%|
|17031320100|170318391

Compare result with last work

In [94]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    #Get  RDDS with the data from the csv file
    trips = sc.textFile('Taxi_Trips_151MB.csv')\
    .map( lambda line : line.split(';'))\
    .map( lambda arr : Row( trip_id = arr[0], taxi_id = arr[1], trip_start_time = arr[2], \
                                               trip_end_time = arr[3], trip_seconds = arr[4], trip_miles = arr[5],\
                           pickup_ID=arr[6], dropoff_ID=arr[7], pickup_com_area=arr[8], dropoff_com_area=arr[9],\
                           fare=arr[10], tips=arr[11], tolls=arr[12], extras=arr[13], trip_total=arr[14], \
                           payment_type=arr[15],company=arr[16], P_C_latitude=arr[17], P_C_longitude=arr[18],\
                           P_C_location=arr[19], D_C_latitude=arr[20], D_C_longitude=arr[21],D_C_location=arr[22] ))
    
    tripsDF = spark.createDataFrame( trips )
    tripsDF.createOrReplaceTempView("trips")
    
    spark.sql('DROP VIEW IF EXISTS count')
    countDF = spark.sql(' CREATE TEMPORARY VIEW count AS SELECT  \
    pickup_ID,dropoff_ID, Count(*) AS Total, AVG(trip_miles) AS avg_miles \
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    spark.sql('DROP VIEW IF EXISTS cash')
    cashDF=spark.sql('CREATE TEMPORARY VIEW cash AS SELECT  \
    pickup_ID,dropoff_ID, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Cash") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Cash\
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    spark.sql('DROP VIEW IF EXISTS credit_card')
    credit_cardDF=spark.sql('CREATE TEMPORARY VIEW credit_card AS SELECT  \
    pickup_ID,dropoff_ID, \
    CONCAT(ROUND(SUM( CASE WHEN (payment_type="Credit Card") THEN 1 ELSE 0 END )/Count(*),2)*100,"%") AS Credit_Card\
    FROM trips\
    WHERE pickup_ID!="" AND dropoff_ID!="" AND trip_miles!="" AND trip_miles!="0"\
    GROUP BY pickup_ID,dropoff_ID')
    
    resultDF=spark.sql('SELECT A.pickup_ID, A.dropoff_ID, A.Total, B.Cash, C.Credit_Card FROM count as A JOIN cash AS B \
    ON A.pickup_ID=B.pickup_ID AND A.dropoff_ID=B.dropoff_ID JOIN credit_card AS C \
    ON A.pickup_ID=C.pickup_ID AND A.dropoff_ID=C.dropoff_ID   ORDER BY A.Total DESC')
    
    
    
    resultDF.show(resultDF.count())
    
    
    
    

    sc.stop()
except:
    sc.stop()

+-----------+-----------+-----+-------------------+-------------------+
|  pickup_ID| dropoff_ID|Total|               Cash|        Credit_Card|
+-----------+-----------+-----+-------------------+-------------------+
|17031839100|17031839100| 4124|              63.0%|              36.0%|
|17031320100|17031839100| 2784| 55.00000000000001%|              44.0%|
|17031839100|17031320100| 2552| 56.99999999999999%|              42.0%|
|17031081500|17031839100| 1960| 56.00000000000001%|              43.0%|
|17031839100|17031281900| 1916|              67.0%|              32.0%|
|17031281900|17031839100| 1890|              64.0%|              35.0%|
|17031839100|17031081500| 1626| 57.99999999999999%|              41.0%|
|17031839100|17031081700| 1609| 55.00000000000001%|              44.0%|
|17031081700|17031839100| 1503| 56.99999999999999%|              42.0%|
|17031980000|17031839100| 1496|              21.0%|              78.0%|
|17031839100|17031081800| 1422|              52.0%|             